In [1]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import pandas as pd
import numpy as np
import matplotlib

In [2]:
# url_imagen = '/home/diego/Proyecto Vivero/data/01_VIVERO/01_imagenes/imagen_mesones_rgb.tif'
# shp = '/home/diego/Proyecto Vivero/data/01_VIVERO/02_puntos_plantas/plantas_mesones_2.shp'
# url_salida = "/home/diego/Proyecto Vivero/train2"
url_imagen = 'D:/Memoria/Codigo/AraucoNursery/PROYECTOS/VIVERO/imagenes/imagen_mesones_rgb.tif'
shp = 'D:/Memoria/Codigo/AraucoNursery/PROYECTOS/VIVERO/puntos_plantas/plantas_mesones_2.shp'
url_salida = "D:/Memoria/Codigo/AraucoNursery/train"

In [3]:
puntos = gpd.read_file(shp)
raster = rasterio.open(url_imagen)

In [ ]:
puntos

In [4]:
offsetX = 0
offsetY = 0
width = 100
height = 100
step = 100
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [23]:
contador = 0

out_meta = raster.meta.copy()
out_meta.update(  {"driver": "GTiff",
                   "height": 100,
                    "width": 100,
                    "transform": out_transform,
                    "crs": raster.crs})


while (1):
        # finalizar si se a recorrido toda la imagen
    if(offsetY >= raster.height and offsetX >= raster.width):
        break
     #cuando llegue al borde en el eje x, debo bajar(sumar en eje y) y debo volver a empezar setear offsetX en 0 y widthX en 100
    elif(offsetX >= raster.width):
        offsetY = offsetY + step
        offsetX = 0
   
    #entregamos los pixeles para obtener las coordenadas
    t = raster.xy(offsetX, offsetY)
    l = raster.xy(offsetX + width , offsetY)
    r = raster.xy(offsetX, offsetY + height)
    b = raster.xy(offsetX + width , offsetY + height)  
    bbox = box(l[0], b[1], r[0], t[1])
        
    try:
        #transformacion de la imagen
        out_img, out_transform = mask(raster, shapes=[bbox], crop=True)
        valores = np.unique(out_img)
        # si solo se tiene valores NODATA en la imagen, la descartamos
        
        if np.array_equal(valores,[65535]):
            offsetX = offsetX + step
            contador = contador + 1
            continue
            
        matplotlib.image.imsave('{}/salida{}.png'.format(url_salida,contador), out_img.T)

    #interseccion ptos
        interseccion = puntos["geometry"].intersects(bbox)
        cantidad = interseccion.value_counts()

        
        if(True in cantidad.index):
            numero = cantidad[True]
            coordenadas = puntos[interseccion]

            with rasterio.open("train/salida"+str(contador)+".tif", "w", **out_meta) as dest:
                dest.write(out_img)

                for element in coordenadas["geometry"]:
                    x, y = element.coords.xy
                    print("coordenadas: ",x[0],y[0])
                    py, px = dest.index(y, x)
                    print("pixeles: ",px,py)
                    
#             print()
        else:
            numero = 0
        
#         cuentas.loc[contador] = ["salida"+str(contador)+".png",numero]
        if(contador%1000 == 0):
            print("salida"+str(contador)+".png")

        
    except ValueError:
        pass
#         print(url_salida+"/salida"+str(contador)+".pngtif")
#         print(coords)
    contador = contador + 1
    offsetX = offsetX + step 
   

coordenadas:  738096.1440252806 6090373.649225203
pixeles:  [483931296] [483931205]
coordenadas:  738096.1329652839 6090373.62710521
pixeles:  [483931294] [483931206]
coordenadas:  738096.1440252806 6090373.582865223
pixeles:  [483931290] [483931205]
coordenadas:  738096.2103852608 6090373.56074523
pixeles:  [483931288] [483931199]
coordenadas:  738095.1375655824 6090373.317425303
pixeles:  [483931266] [483931296]
coordenadas:  738095.1596855758 6090373.317425303
pixeles:  [483931266] [483931294]
coordenadas:  738095.1596855758 6090373.29530531
pixeles:  [483931264] [483931294]
coordenadas:  738095.1818055691 6090373.29530531
pixeles:  [483931264] [483931292]
coordenadas:  738095.2149855591 6090373.284245313
pixeles:  [483931263] [483931289]
coordenadas:  738095.2371055526 6090373.26212532
pixeles:  [483931261] [483931287]
coordenadas:  738095.2039255624 6090373.251065323
pixeles:  [483931260] [483931290]
coordenadas:  738095.2592255459 6090373.251065323
pixeles:  [483931260] [48393128

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


coordenadas:  738095.5025454729 6090371.448285863
pixeles:  [483931097] [483931263]
coordenadas:  738095.5578454564 6090371.448285863
pixeles:  [483931097] [483931258]
coordenadas:  738095.8343453734 6090371.448285863
pixeles:  [483931097] [483931233]
coordenadas:  738096.0002453238 6090371.448285863
pixeles:  [483931097] [483931218]
coordenadas:  738095.6020854431 6090371.437225866
pixeles:  [483931096] [483931254]
coordenadas:  738095.6684454232 6090371.437225866
pixeles:  [483931096] [483931248]
coordenadas:  738096.1550852773 6090371.437225866
pixeles:  [483931096] [483931204]
coordenadas:  738095.4583054862 6090371.426165869
pixeles:  [483931095] [483931267]
coordenadas:  738095.5467854597 6090371.426165869
pixeles:  [483931095] [483931259]
coordenadas:  738095.7569253967 6090371.426165869
pixeles:  [483931095] [483931240]
coordenadas:  738095.7901053868 6090371.426165869
pixeles:  [483931095] [483931237]
coordenadas:  738095.9449453403 6090371.426165869
pixeles:  [483931095] [483

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


 [483931208]
coordenadas:  738095.3034655326 6090370.607726115
pixeles:  [483931021] [483931281]
coordenadas:  738095.3366455226 6090370.607726115
pixeles:  [483931021] [483931278]
coordenadas:  738095.6795054199 6090370.607726115
pixeles:  [483931021] [483931247]
coordenadas:  738095.7016254133 6090370.607726115
pixeles:  [483931021] [483931245]
coordenadas:  738095.8564653668 6090370.607726115
pixeles:  [483931021] [483931231]
coordenadas:  738095.956005337 6090370.607726115
pixeles:  [483931021] [483931222]
coordenadas:  738096.166145274 6090370.607726115
pixeles:  [483931021] [483931203]
coordenadas:  738095.2039255624 6090370.596666118
pixeles:  [483931020] [483931290]
coordenadas:  738095.6131454398 6090370.596666118
pixeles:  [483931020] [483931253]
coordenadas:  738095.7348054033 6090370.596666118
pixeles:  [483931020] [483931242]
coordenadas:  738095.1265055856 6090370.585606121
pixeles:  [483931019] [483931297]
coordenadas:  738095.4251254961 6090370.585606121
pixeles:  [4839

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


coordenadas:  738095.3477055194 6090370.342286195
pixeles:  [483930997] [483931277]
coordenadas:  738095.3698255128 6090370.342286195
pixeles:  [483930997] [483931275]
coordenadas:  738095.6463254298 6090370.342286195
pixeles:  [483930997] [483931250]
coordenadas:  738095.7901053868 6090370.342286195
pixeles:  [483930997] [483931237]
coordenadas:  738095.9449453403 6090370.342286195
pixeles:  [483930997] [483931223]
coordenadas:  738096.0002453238 6090370.342286195
pixeles:  [483930997] [483931218]
coordenadas:  738096.0997852939 6090370.342286195
pixeles:  [483930997] [483931209]
coordenadas:  738095.1596855758 6090370.331226198
pixeles:  [483930996] [483931294]
coordenadas:  738095.7016254133 6090370.331226198
pixeles:  [483930996] [483931245]
coordenadas:  738096.0555453071 6090370.331226198
pixeles:  [483930996] [483931213]
coordenadas:  738096.1550852773 6090370.331226198
pixeles:  [483930996] [483931204]
coordenadas:  738095.2039255624 6090370.320166201
pixeles:  [483930995] [483

KeyboardInterrupt: 

In [51]:
cuentas.to_csv(url_salida+"/cuentas.csv")